In [1]:
pip install sacrebleu

  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/de/ea/025db0a39337b63d4728a900d262c39c3029b0fe76a9876ce6297b1aa6a0/sacrebleu-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.0 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
import logging
import os
import pathlib
import random
import sys
sys.path.append('/kaggle/input/supporting-files')
import click

import sacrebleu
import torch
import torch.nn as nn
import tqdm
import config
import corpora
import utils
from main_fix2 import Network as FixNN
from main_para import EncoderRNN as ParEncNN
from main_para import AttnDecoderRNN as ParDecNN

In [3]:
#pip install config

In [16]:
	#qqp sentences:

debug_sentences = [
    'How do I get funding for my web based startup idea ?',
    'What do intelligent people do to pass time ?',
    'Which is the best SEO Company in Delhi ?',
    'Why do you waer makeup ?',
    'How do start chatting with a girl ?',
    'What is the meaning of living life ?',
    'Why do my armpits hurt ?',
    'Why does eye color change with age ?',
    'How do you find the standard deviation of a probability distribution ? What are some examples ?',
    'How can I complete my 11 syllabus in one month ?',
    'How do I concentrate better on my studies ?',
    'Which is the best retirement plan in india ?',
    'Should I tell my best friend I love her ?',
    'Which is the best company for Appian Vagrant online job support ?',
    'How can one do for good handwriting ?',
    'What are remedies to get rid of belly fat ?',
    'What is the best way to cook precooked turkey ?',
    'What is the future of e-commerce in India ?',
    'Why do my burps taste like rotten eggs ?',
    'What is an example of chemical weathering ?',
    'What are some of the advantages and disadvantages of cyber schooling ?',
    'How can I increase traffic to my websites by Facebook ?',
    'How do I increase my patience level in life ?',
    'What are the best hospitals for treating cancer in India ?',
    'Will Jio sim work in a 3G phone ? If yes , how ?',
]

debug_sentences = [s.split(" ") for s in debug_sentences]

In [17]:
# Get the current working directory
cwd = os.getcwd()
print(cwd)

/kaggle/working


In [18]:
cwd = os.path.dirname('/kaggle/working/')
logger = logging.getLogger("main")

In [19]:
class Network(nn.Module):
    def __init__(
        self,
        word2index,
        embeddings,
    ):
        super().__init__()
        self.logger = logging.getLogger(f"{__name__}")
        self.word2index = word2index
        self.index2word = {i: k for k, i in word2index.items()}
        self.fix_gen = FixNN(
            embedding_type="glove",
            vocab_size=len(word2index),
            embedding_dim=config.embedding_dim,
            embeddings=embeddings,
            dropout=config.fix_dropout,
            hidden_dim=config.fix_hidden_dim,
        )
        self.par_enc = ParEncNN(
            input_size=config.embedding_dim,
            hidden_size=config.par_hidden_dim,
            embeddings=embeddings,
        )
        self.par_dec = ParDecNN(
            input_size=config.embedding_dim,
            hidden_size=config.par_hidden_dim,
            output_size=len(word2index),
            embeddings=embeddings,
            dropout_p=config.par_dropout,
            max_length=config.max_length,
        )

    def forward(self, x, target=None, teacher_forcing_ratio=None):
        teacher_forcing_ratio = teacher_forcing_ratio if teacher_forcing_ratio is not None else config.teacher_forcing_ratio
        x1 = nn.utils.rnn.pad_sequence(x, batch_first=True)
        x2 = nn.utils.rnn.pad_sequence(x, batch_first=False)
        fixations = torch.sigmoid(self.fix_gen(x1, [len(_x) for _x in x1]))

        enc_hidden = self.par_enc.initHidden().to(config.DEV)
        enc_outs = torch.zeros(config.max_length, config.par_hidden_dim, device=config.DEV)

        for ei in range(len(x2)):
            enc_out, enc_hidden = self.par_enc(x2[ei], enc_hidden)
            enc_outs[ei] += enc_out[0, 0]

        dec_in = torch.tensor([[self.word2index[config.SOS]]], device=config.DEV)  # SOS
        dec_hidden = enc_hidden
        dec_outs = []
        dec_words = []
        dec_atts = torch.zeros(config.max_length, config.max_length)

        if target is not None:  # training
            target = nn.utils.rnn.pad_sequence(target, batch_first=False)
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                for di in range(len(target)):
                    dec_out, dec_hidden, dec_att = self.par_dec(
                        dec_in, dec_hidden, enc_outs, fixations
                    )
                    dec_outs.append(dec_out)
                    dec_atts[di] = dec_att.data
                    dec_input = target[di]

            else:
                for di in range(len(target)):
                    dec_out, dec_hidden, dec_att = self.par_dec(
                        dec_in, dec_hidden, enc_outs, fixations
                    )
                    dec_outs.append(dec_out)
                    dec_atts[di] = dec_att.data
                    topv, topi = dec_out.data.topk(1)
                    dec_words.append(self.index2word[topi.item()])

                    dec_input = topi.squeeze().detach()

        else:  # prediction
            for di in range(config.max_length):
                dec_out, dec_hidden, dec_att = self.par_dec(
                    dec_in, dec_hidden, enc_outs, fixations
                )
                dec_outs.append(dec_out)
                dec_atts[di] = dec_att.data
                topv, topi = dec_out.data.topk(1)
                if topi.item() == self.word2index[config.EOS]:
                    dec_words.append("<__EOS__>")
                    break
                else:
                    dec_words.append(self.index2word[topi.item()])

                dec_input = topi.squeeze().detach()

        return dec_outs, dec_words, dec_atts[: di + 1], fixations

In [20]:
def load_corpus(corpus_name, splits):
    if not splits:
        return

    logger.info("loading corpus")
    if corpus_name == "msrpc":
        load_fn = corpora.load_msrpc
    elif corpus_name == "qqp":
        load_fn = corpora.load_qqp
    elif corpus_name == "wiki":
        load_fn = corpora.load_wiki
    elif corpus_name == "qqp_paws":
        load_fn = corpora.load_qqp_paws
    elif corpus_name == "qqp_kag":
        load_fn = corpora.load_qqp_kag
    elif corpus_name == "sentiment":
        load_fn = corpora.load_sentiment
    elif corpus_name == "stanford":
        load_fn = corpora.load_stanford
    elif corpus_name == "stanford_sent":
        load_fn = corpora.load_stanford_sent
    elif corpus_name == "tamil":
        load_fn = corpora.load_tamil
    elif corpus_name == "compression":
        load_fn = corpora.load_compression

    corpus = {}
    langs = []

    if "train" in splits:
        train_pairs, train_lang = load_fn("train")
        corpus["train"] = train_pairs
        langs.append(train_lang)
    if "val" in splits:
        val_pairs, val_lang = load_fn("val")
        corpus["val"] = val_pairs
        langs.append(val_lang)
    if "test" in splits:
        test_pairs, test_lang = load_fn("test")
        corpus["test"] = test_pairs
        langs.append(test_lang)

    logger.info("creating word index")
    lang = langs[0]
    for _lang in langs[1:]:
        lang += _lang
    word2index = lang.word2index

    index2word = {i: w for w, i in word2index.items()}

    return corpus, word2index, index2word


def init_network(word2index):
    logger.info("loading embeddings")
    vocabulary = sorted(word2index.keys())
    embeddings = utils.load_glove(vocabulary)

    logger.info("initializing model")
    network = Network(
        word2index=word2index,
        embeddings=embeddings,
    )
    network.to(config.DEV)

    print(f"#parameters: {sum(p.numel() for p in network.parameters())}")

    return network

In [21]:
@click.group(context_settings=dict(help_option_names=["-h", "--help"]))
@click.option("-v", "--verbose", count=True)
@click.option("-d", "--debug", is_flag=True)
def main(verbose, debug):
    if verbose == 0:
        loglevel = logging.ERROR
    elif verbose == 1:
        loglevel = logging.WARN
    elif verbose >= 2:
        loglevel = logging.INFO

    if debug:
        loglevel = logging.DEBUG

    logging.basicConfig(
        format="[%(asctime)s] <%(name)s> %(levelname)s: %(message)s",
        datefmt="%d.%m. %H:%M:%S",
        level=loglevel,
    )

    logger.debug("arguments: %s" % str(sys.argv))

In [22]:
def train(corpus_name, model_name, bleu, fixation_weights=None, freeze_fixations=False):
    corpus, word2index, index2word = load_corpus(corpus_name, ["train", "val"])
    train_pairs = corpus["train"]
    val_pairs = corpus["val"]
    network = init_network(word2index)

    model_dir = os.path.join("models", model_name)
    logger.debug("creating model dir %s" % model_dir)
    pathlib.Path(model_dir).mkdir(parents=True)

    if fixation_weights is not None:
        logger.info("loading fixation prediction checkpoint")
        checkpoint = torch.load(fixation_weights, map_location=config.DEV)
        if "word2index" in checkpoint:
            weights = checkpoint["weights"]
        else:
            weights = checkpoint

        # remove the embedding layer before loading
        weights = {k: v for k, v in weights.items() if not k.startswith("pre.embedding_layer")}
        network.fix_gen.load_state_dict(weights, strict=False)

        if freeze_fixations:
            logger.info("freezing fixation generation network")
            for p in network.fix_gen.parameters():
                p.requires_grad = False

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=config.learning_rate)
    #optimizer = torch.optim.Adam(network.parameters(), lr=1e-4, weight_decay=1e-5)

    best_val_loss = None

    epoch = 1
    
    while epoch<6:
        train_batch_iter = utils.pair_iter(pairs=train_pairs, word2index=word2index, shuffle=True, shuffle_pairs=False)
        val_batch_iter = utils.pair_iter(pairs=val_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)
        # test_batch_iter = utils.pair_iter(pairs=test_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

        running_train_loss = 0
        total_train_loss = 0
        total_val_loss = 0

        if bleu == "sacrebleu":
            running_train_bleu = 0
            total_train_bleu = 0
            total_val_bleu = 0
        elif bleu == "nltk":
            running_train_bleu_1 = 0
            running_train_bleu_2 = 0
            running_train_bleu_3 = 0
            running_train_bleu_4 = 0
            total_train_bleu_1 = 0
            total_train_bleu_2 = 0
            total_train_bleu_3 = 0
            total_train_bleu_4 = 0
            total_val_bleu_1 = 0
            total_val_bleu_2 = 0
            total_val_bleu_3 = 0
            total_val_bleu_4 = 0

        network.train()
        for i, batch in enumerate(train_batch_iter, 1):
            optimizer.zero_grad()

            input, target,rating = batch
            prediction, words, attention, fixations = network(input, target)

            loss = loss_fn(
                torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
            )

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            total_train_loss += loss.item()

            _prediction = " ".join([index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()])
            _target = [index2word[_x] for _x in target[0].tolist()]

            if bleu == "sacrebleu":
                bleu_score = sacrebleu.sentence_bleu(_prediction, _target).score
                running_train_bleu += bleu_score
                total_train_bleu += bleu_score
            elif bleu == "nltk":
                bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
                bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
                bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
                bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
                running_train_bleu_1 += bleu_1_score
                running_train_bleu_2 += bleu_2_score
                running_train_bleu_3 += bleu_3_score
                running_train_bleu_4 += bleu_4_score
                total_train_bleu_1 += bleu_1_score
                total_train_bleu_2 += bleu_2_score
                total_train_bleu_3 += bleu_3_score
                total_train_bleu_4 += bleu_4_score
            # print(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist())

            if i % 1000 == 0:

                if bleu == "sacrebleu":

                    print(f"step {i} avg_train_loss {running_train_loss/100:.4f} avg_train_bleu {running_train_bleu/100:.2f}")
                elif bleu == "nltk":
                    print(f"step {i} avg_train_loss {running_train_loss/100:.4f} avg_train_bleu_1 {running_train_bleu_1/100:.2f} avg_train_bleu_2 {running_train_bleu_2/100:.2f} avg_train_bleu_3 {running_train_bleu_3/100:.2f} avg_train_bleu_4 {running_train_bleu_4/100:.2f}")

                network.eval()
                with open(os.path.join(model_dir, f"debug_{epoch}_{i}.out"), "w") as h:
                    if bleu == "sacrebleu":
                        h.write(f"# avg_train_loss {running_train_loss/100:.4f} avg_train_bleu {running_train_bleu/100:.2f}")
                        running_train_bleu = 0
                    elif bleu == "nltk":
                        h.write(f"# avg_train_loss {running_train_loss/100:.4f} avg_train_bleu_1 {running_train_bleu_1/100:.2f} avg_train_bleu_2 {running_train_bleu_2/100:.2f} avg_train_bleu_3 {running_train_bleu_3/100:.2f} avg_train_bleu_4 {running_train_bleu_4/100:.2f}")
                        running_train_bleu_1 = 0
                        running_train_bleu_2 = 0
                        running_train_bleu_3 = 0
                        running_train_bleu_4 = 0

                    running_train_loss = 0

                    h.write("\n")
                    h.write("\t".join(["sentence", "prediction", "attention", "fixations"]))
                    h.write("\n")
                    for s, input in zip(debug_sentences, utils.sent_iter(debug_sentences, word2index=word2index)):
                        prediction, words, attentions, fixations = network(input)
                        prediction = torch.argmax(torch.stack(prediction).squeeze(1), -1).detach().cpu().tolist()
                        prediction = [index2word.get(x, "<__UNK__>") for x in prediction]
                        attentions = attentions.detach().cpu().squeeze().tolist()
                        fixations = fixations.detach().cpu().squeeze().tolist()
                        h.write(f"{s}\t{prediction}\t{attentions}\t{fixations}")
                        h.write("\n")

                network.train()

        network.eval()
        for i, batch in enumerate(val_batch_iter):
            input, target ,rating = batch
            prediction, words, attention, fixations = network(input, target, teacher_forcing_ratio=0)
            loss = loss_fn(
                torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
            )

            _prediction = " ".join([index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()])
            _target = [index2word[_x] for _x in target[0].tolist()]
            if bleu == "sacrebleu":
                bleu_score = sacrebleu.sentence_bleu(_prediction, _target).score
                total_val_bleu += bleu_score
            elif bleu == "nltk":
                bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
                bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
                bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
                bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
                total_val_bleu_1 += bleu_1_score
                total_val_bleu_2 += bleu_2_score
                total_val_bleu_3 += bleu_3_score
                total_val_bleu_4 += bleu_4_score

            total_val_loss += loss.item()

        avg_val_loss = total_val_loss/len(val_pairs)

        if bleu == "sacrebleu":
            print(f"epoch {epoch} avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu {total_train_bleu/len(train_pairs):.2f} avg_val_bleu {total_val_bleu/len(val_pairs):.2f}")
        elif bleu == "nltk":
            print(f"epoch {epoch} avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu_1 {total_train_bleu_1/len(train_pairs):.2f} avg_train_bleu_2 {total_train_bleu_2/len(train_pairs):.2f} avg_train_bleu_3 {total_train_bleu_3/len(train_pairs):.2f} avg_train_bleu_4 {total_train_bleu_4/len(train_pairs):.2f} avg_val_bleu_1 {total_val_bleu_1/len(val_pairs):.2f} avg_val_bleu_2 {total_val_bleu_2/len(val_pairs):.2f} avg_val_bleu_3 {total_val_bleu_3/len(val_pairs):.2f} avg_val_bleu_4 {total_val_bleu_4/len(val_pairs):.2f}")

        with open(os.path.join(model_dir, f"debug_{epoch}_end.out"), "w") as h:
            if bleu == "sacrebleu":
                h.write(f"# avg_train_loss {total_train_loss/len(train_pairs)} avg_val_loss {total_val_loss/len(val_pairs)} avg_train_bleu {total_train_bleu/len(train_pairs)} avg_val_bleu {total_val_bleu/len(val_pairs)}")
            elif bleu == "nltk":
                h.write(f"# avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu_1 {total_train_bleu_1/len(train_pairs):.2f} avg_train_bleu_2 {total_train_bleu_2/len(train_pairs):.2f} avg_train_bleu_3 {total_train_bleu_3/len(train_pairs):.2f} avg_train_bleu_4 {total_train_bleu_4/len(train_pairs):.2f} avg_val_bleu_1 {total_val_bleu_1/len(val_pairs):.2f} avg_val_bleu_2 {total_val_bleu_2/len(val_pairs):.2f} avg_val_bleu_3 {total_val_bleu_3/len(val_pairs):.2f} avg_val_bleu_4 {total_val_bleu_4/len(val_pairs):.2f}")
            h.write("\n")
            h.write("\t".join(["sentence", "prediction", "attention", "fixations"]))
            h.write("\n")
            for s, input in zip(debug_sentences, utils.sent_iter(debug_sentences, word2index=word2index)):
                prediction, words, attentions, fixations = network(input)
                prediction = torch.argmax(torch.stack(prediction).squeeze(1), -1).detach().cpu().tolist()
                prediction = [index2word.get(x, "<__UNK__>") for x in prediction]
                attentions = attentions.detach().cpu().squeeze().tolist()
                fixations = fixations.detach().cpu().squeeze().tolist()
                h.write(f"{s}\t{prediction}\t{attentions}\t{fixations}")
                h.write("\n")
                
        utils.save_model(network, word2index, os.path.join(model_dir, f"{model_name}_{epoch}"))

        if best_val_loss is None or avg_val_loss < best_val_loss:
            if best_val_loss is not None:
                logger.info(f"{avg_val_loss} < {best_val_loss} ({avg_val_loss-best_val_loss}): new best model from epoch {epoch}")
            else:
                logger.info(f"{avg_val_loss} < {best_val_loss}: new best model from epoch {epoch}")

            best_val_loss = avg_val_loss
            # save_model(model, word2index, model_name + "_epoch_" + str(epoch))
            # utils.save_model(network, word2index, os.path.join(model_dir, f"{model_name}_best"))

        epoch += 1

In [23]:
def val(corpus_name, model_weights, sentence_statistics, bleu):
    corpus, word2index, index2word = load_corpus(corpus_name, ["val"])
    val_pairs = corpus["val"]

    logger.info("loading model checkpoint")
    checkpoint = torch.load(model_weights, map_location=config.DEV)
    if "word2index" in checkpoint:
        weights = checkpoint["weights"]
        word2index = checkpoint["word2index"]
        index2word = {i: w for w, i in word2index.items()}
    else:
        asdf

    network = init_network(word2index)

    # remove the embedding layer before loading
    weights = {k: v for k, v in weights.items() if not "embedding" in k}
    # make a new output layer to match the weights from the checkpoint
    # we cannot remove it like we did with the embedding layers because
    # unlike those the output layer actually contains learned parameters
    vocab_size, hidden_size = weights["par_dec.out.weight"].shape
    network.par_dec.out = nn.Linear(hidden_size, vocab_size).to(config.DEV)
    # actually load the parameters
    network.load_state_dict(weights, strict=False)

    loss_fn = nn.CrossEntropyLoss()

    val_batch_iter = utils.pair_iter(pairs=val_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

    total_val_loss = 0
    if bleu == "sacrebleu":
        total_val_bleu = 0
    elif bleu == "nltk":
        total_val_bleu_1 = 0
        total_val_bleu_2 = 0
        total_val_bleu_3 = 0
        total_val_bleu_4 = 0

    network.eval()
    for i, batch in enumerate(val_batch_iter, 1):
        input, target = batch
        prediction, words, attentions, fixations = network(input, target)

        loss = loss_fn(
            torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
        )
        total_val_loss += loss.item()

        _prediction = [index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()]
        _target = [index2word[_x] for _x in target[0].tolist()]
        if bleu == "sacrebleu":
            bleu_score = sacrebleu.sentence_bleu(" ".join(_prediction), " ".join(_target)).score
            total_val_bleu += bleu_score
        elif bleu == "nltk":
            bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
            bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
            bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
            bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
            total_val_bleu_1 += bleu_1_score
            total_val_bleu_2 += bleu_2_score
            total_val_bleu_3 += bleu_3_score
            total_val_bleu_4 += bleu_4_score

        if sentence_statistics:
            s = [index2word[x] for x in input[0].detach().cpu().tolist()]
            attentions = attentions.detach().cpu().squeeze().tolist()
            fixations = fixations.detach().cpu().squeeze().tolist()

            if bleu == "sacrebleu":
                print(f"{bleu_score}\t{s}\t{_prediction}\t{_target}\t{attentions}\t{fixations}")
            elif bleu == "nltk":
                print(f"{bleu_1_score}\t{bleu_2_score}\t{bleu_3_score}\t{bleu_4_score}\t{s}\t{_prediction}\t{_target}\t{attentions}\t{fixations}")

    if bleu == "sacrebleu":
        print(f"avg_val_loss {total_val_loss/len(val_pairs):.4f} avg_val_bleu {total_val_bleu/len(val_pairs):.2f}")
    elif bleu == "nltk":
        print(f"avg_val_loss {total_val_loss/len(val_pairs):.4f} avg_val_bleu_1 {total_val_bleu_1/len(val_pairs):.2f} avg_val_bleu_2 {total_val_bleu_2/len(val_pairs):.2f} avg_val_bleu_3 {total_val_bleu_3/len(val_pairs):.2f} avg_val_bleu_4 {total_val_bleu_4/len(val_pairs):.2f}")

In [24]:
def test(corpus_name, model_weights, sentence_statistics, bleu):
    corpus, word2index, index2word = load_corpus(corpus_name, ["test"])
    test_pairs = corpus["test"]

    if model_weights is not None:
        logger.info("loading model checkpoint")
        checkpoint = torch.load(model_weights, map_location=config.DEV)
        if "word2index" in checkpoint:
            weights = checkpoint["weights"]
            word2index = checkpoint["word2index"]
            index2word = {i: w for w, i in word2index.items()}
        else:
            asdf

    network = init_network(word2index)

    if model_weights is not None:
        # remove the embedding layer before loading
        weights = {k: v for k, v in weights.items() if not "embedding" in k}
        # make a new output layer to match the weights from the checkpoint
        # we cannot remove it like we did with the embedding layers because
        # unlike those the output layer actually contains learned parameters
        vocab_size, hidden_size = weights["par_dec.out.weight"].shape
        network.par_dec.out = nn.Linear(hidden_size, vocab_size).to(config.DEV)
        # actually load the parameters
        network.load_state_dict(weights, strict=False)

    loss_fn = nn.CrossEntropyLoss()

    test_batch_iter = utils.pair_iter(pairs=test_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

    total_test_loss = 0
    if bleu == "sacrebleu":
        total_test_bleu = 0
    elif bleu == "nltk":
        total_test_bleu_1 = 0
        total_test_bleu_2 = 0
        total_test_bleu_3 = 0
        total_test_bleu_4 = 0

    network.eval()
    for i, batch in enumerate(test_batch_iter, 1):
        input, target = batch

        prediction, words, attentions, fixations = network(input, target)

        loss = loss_fn(
            torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
        )
        total_test_loss += loss.item()

        _prediction = [index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()]
        _target = [index2word[_x] for _x in target[0].tolist()]
        if bleu == "sacrebleu":
            bleu_score = sacrebleu.sentence_bleu(" ".join(_prediction), " ".join( _target)).score
            total_test_bleu += bleu_score
        elif bleu == "nltk":
            bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
            bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
            bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
            bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
            total_test_bleu_1 += bleu_1_score
            total_test_bleu_2 += bleu_2_score
            total_test_bleu_3 += bleu_3_score
            total_test_bleu_4 += bleu_4_score

        if sentence_statistics:
            s = [index2word[x] for x in input[0].detach().cpu().tolist()]
            attentions = attentions.detach().cpu().squeeze().tolist()
            fixations = fixations.detach().cpu().squeeze().tolist()

            if bleu == "sacrebleu":
                print(f"{bleu_score}\t{s}\t{_prediction}\t{attentions}\t{fixations}")
            elif bleu == "nltk":
                print(f"{bleu_1_score}\t{bleu_2_score}\t{bleu_3_score}\t{bleu_4_score}\t{s}\t{_prediction}\t{attentions}\t{fixations}")

    if bleu == "sacrebleu":
        print(f"avg_test_loss {total_test_loss/len(test_pairs):.4f} avg_test_bleu {total_test_bleu/len(test_pairs):.2f}")
    elif bleu == "nltk":
        print(f"avg_test_loss {total_test_loss/len(test_pairs):.4f} avg_test_bleu_1 {total_test_bleu_1/len(test_pairs):.2f} avg_test_bleu_2 {total_test_bleu_2/len(test_pairs):.2f} avg_test_bleu_3 {total_test_bleu_3/len(test_pairs):.2f} avg_test_bleu_4 {total_test_bleu_4/len(test_pairs):.2f}")

In [13]:
def predict(corpus_name, model_weights):
    corpus, word2index, index2word = load_corpus(corpus_name, ["val"])
    test_pairs = corpus["val"]
    s_list = []
    fixations_list = []

    if model_weights is not None:
        logger.info("loading model checkpoint")
        checkpoint = torch.load(model_weights, map_location=config.DEV)
        if "word2index" in checkpoint:
            weights = checkpoint["weights"]
            word2index = checkpoint["word2index"]
            index2word = {i: w for w, i in word2index.items()}
        else:
            asdf

    network = init_network(word2index)

    logger.info(f"vocab size {len(word2index)}")

    if model_weights is not None:
        # remove the embedding layer before loading
        weights = {k: v for k, v in weights.items() if "embedding" not in k}
        # make a new output layer to match the weights from the checkpoint
        # we cannot remove it like we did with the embedding layers because
        # unlike those the output layer actually contains learned parameters
        vocab_size, hidden_size = weights["par_dec.out.weight"].shape
        network.par_dec.out = nn.Linear(hidden_size, vocab_size).to(config.DEV)
        # actually load the parameters
        network.load_state_dict(weights, strict=False)

    test_batch_iter = utils.pair_iter(pairs=test_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

    network.eval()
    for i, batch in enumerate(test_batch_iter, 1):
        if i > 5:
            break  # exit the loop after 5 iterations

        input, target, rating = batch

        prediction, words, attentions, fixations = network(input, target)
        _prediction = [index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1), -1).tolist()]

        s = [index2word[x] for x in input[0].detach().cpu().tolist()]
        attentions = attentions.detach().cpu().squeeze().tolist()
        fixations = fixations.detach().cpu().squeeze().tolist()

        s_list.append(s)
        fixations_list.append(fixations)

    return s_list, fixations_list

# Example usage:
s_list, fixations_list = predict("qqp", "/kaggle/input/supporting-files/my_model_12.tar")


128 4
#parameters: 89921670


In [25]:
def predict1(corpus_name, model_weights):
    corpus, word2index, index2word = load_corpus(corpus_name, ["val"])
    test_pairs = corpus["val"]

    if model_weights is not None:
        logger.info("loading model checkpoint")
        checkpoint = torch.load(model_weights, map_location=config.DEV)
        if "word2index" in checkpoint:
            weights = checkpoint["weights"]
            word2index = checkpoint["word2index"]
            index2word = {i: w for w, i in word2index.items()}
        else:
            asdf

    network = init_network(word2index)

    logger.info(f"vocab size {len(word2index)}")
    
    if model_weights is not None:
        # remove the embedding layer before loading
        weights = {k: v for k, v in weights.items() if not "embedding" in k}
        # make a new output layer to match the weights from the checkpoint
        # we cannot remove it like we did with the embedding layers because
        # unlike those the output layer actually contains learned parameters
        vocab_size, hidden_size = weights["par_dec.out.weight"].shape
        network.par_dec.out = nn.Linear(hidden_size, vocab_size).to(config.DEV)
        # actually load the parameters
        network.load_state_dict(weights, strict=False)

    test_batch_iter = utils.pair_iter(pairs=test_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

    network.eval()
    for i, batch in enumerate(test_batch_iter, 1):
        if i>5:
            break
        input, target, rating = batch

        prediction, words, attentions, fixations = network(input, target)
        _prediction = [index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1), -1).tolist()]

        s = [index2word[x] for x in input[0].detach().cpu().tolist()]
        attentions = attentions.detach().cpu().squeeze().tolist()
        fixations = fixations.detach().cpu().squeeze().tolist()

        print(f"{s}\t{_prediction}\t{attentions}\t{fixations}")

In [18]:
import os
os.remove("/kaggle/working/glove.cache")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/glove.cache'

In [14]:
def re_train(corpus_name, model_name, bleu, fixation_weights=None, freeze_fixations=False):
    corpus, word2index, index2word = load_corpus(corpus_name, ["train", "val"])
    train_pairs = corpus["train"]
    val_pairs = corpus["val"]
    network = init_network(word2index)

    model_dir = os.path.join("models", model_name)
    logger.debug("creating model dir %s" % model_dir)
    pathlib.Path(model_dir).mkdir(parents=True)

    if fixation_weights is not None:
        logger.info("loading fixation prediction checkpoint")
        checkpoint = torch.load(fixation_weights, map_location=config.DEV)
        if "word2index" in checkpoint:
            weights = checkpoint["weights"]
        else:
            weights = checkpoint

        # remove the embedding layer before loading
        weights = {k: v for k, v in weights.items() if not k.startswith("pre.embedding_layer")}
        network.fix_gen.load_state_dict(weights, strict=False)

        if freeze_fixations:
            logger.info("freezing fixation generation network")
            for p in network.fix_gen.parameters():
                p.requires_grad = False

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=config.learning_rate)
    #optimizer = torch.optim.Adam(network.parameters(), lr=1e-4, weight_decay=1e-5)

    epoch = 3
    # Load previously saved model and word2index
    checkpoint_path = '/kaggle/input/supporting-files/my_model_3.tar'
    checkpoint = torch.load(checkpoint_path, map_location=config.DEV)
    network.load_state_dict(checkpoint["weights"])

    word2index = checkpoint["word2index"]
    best_val_loss = 5.471
    
    while epoch<8:
        train_batch_iter = utils.pair_iter(pairs=train_pairs, word2index=word2index, shuffle=True, shuffle_pairs=False)
        val_batch_iter = utils.pair_iter(pairs=val_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)
        # test_batch_iter = utils.pair_iter(pairs=test_pairs, word2index=word2index, shuffle=False, shuffle_pairs=False)

        running_train_loss = 0
        total_train_loss = 0
        total_val_loss = 0

        if bleu == "sacrebleu":
            running_train_bleu = 0
            total_train_bleu = 0
            total_val_bleu = 0
        elif bleu == "nltk":
            running_train_bleu_1 = 0
            running_train_bleu_2 = 0
            running_train_bleu_3 = 0
            running_train_bleu_4 = 0
            total_train_bleu_1 = 0
            total_train_bleu_2 = 0
            total_train_bleu_3 = 0
            total_train_bleu_4 = 0
            total_val_bleu_1 = 0
            total_val_bleu_2 = 0
            total_val_bleu_3 = 0
            total_val_bleu_4 = 0

        network.train()
        for i, batch in enumerate(train_batch_iter, 1):
            optimizer.zero_grad()

            input, target,rating = batch
            prediction, words, attention, fixations = network(input, target)

            loss = loss_fn(
                torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
            )

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
            total_train_loss += loss.item()

            _prediction = " ".join([index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()])
            _target = [index2word[_x] for _x in target[0].tolist()]

            if bleu == "sacrebleu":
                bleu_score = sacrebleu.sentence_bleu(_prediction, _target).score
                running_train_bleu += bleu_score
                total_train_bleu += bleu_score
            elif bleu == "nltk":
                bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
                bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
                bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
                bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
                running_train_bleu_1 += bleu_1_score
                running_train_bleu_2 += bleu_2_score
                running_train_bleu_3 += bleu_3_score
                running_train_bleu_4 += bleu_4_score
                total_train_bleu_1 += bleu_1_score
                total_train_bleu_2 += bleu_2_score
                total_train_bleu_3 += bleu_3_score
                total_train_bleu_4 += bleu_4_score
            # print(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist())

            if i % 1000 == 0:

                if bleu == "sacrebleu":

                    print(f"step {i} avg_train_loss {running_train_loss/100:.4f} avg_train_bleu {running_train_bleu/100:.2f}")
                elif bleu == "nltk":
                    print(f"step {i} avg_train_loss {running_train_loss/100:.4f} avg_train_bleu_1 {running_train_bleu_1/100:.2f} avg_train_bleu_2 {running_train_bleu_2/100:.2f} avg_train_bleu_3 {running_train_bleu_3/100:.2f} avg_train_bleu_4 {running_train_bleu_4/100:.2f}")

                network.eval()
                with open(os.path.join(model_dir, f"debug_{epoch}_{i}.out"), "w") as h:
                    if bleu == "sacrebleu":
                        h.write(f"# avg_train_loss {running_train_loss/100:.4f} avg_train_bleu {running_train_bleu/100:.2f}")
                        running_train_bleu = 0
                    elif bleu == "nltk":
                        h.write(f"# avg_train_loss {running_train_loss/100:.4f} avg_train_bleu_1 {running_train_bleu_1/100:.2f} avg_train_bleu_2 {running_train_bleu_2/100:.2f} avg_train_bleu_3 {running_train_bleu_3/100:.2f} avg_train_bleu_4 {running_train_bleu_4/100:.2f}")
                        running_train_bleu_1 = 0
                        running_train_bleu_2 = 0
                        running_train_bleu_3 = 0
                        running_train_bleu_4 = 0

                    running_train_loss = 0

                    h.write("\n")
                    h.write("\t".join(["sentence", "prediction", "attention", "fixations"]))
                    h.write("\n")
                    for s, input in zip(debug_sentences, utils.sent_iter(debug_sentences, word2index=word2index)):
                        prediction, words, attentions, fixations = network(input)
                        prediction = torch.argmax(torch.stack(prediction).squeeze(1), -1).detach().cpu().tolist()
                        prediction = [index2word.get(x, "<__UNK__>") for x in prediction]
                        attentions = attentions.detach().cpu().squeeze().tolist()
                        fixations = fixations.detach().cpu().squeeze().tolist()
                        h.write(f"{s}\t{prediction}\t{attentions}\t{fixations}")
                        h.write("\n")

                network.train()

        network.eval()
        for i, batch in enumerate(val_batch_iter):
            input, target ,rating = batch
            prediction, words, attention, fixations = network(input, target, teacher_forcing_ratio=0)
            loss = loss_fn(
                torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], target[0]
            )

            _prediction = " ".join([index2word[_x] for _x in torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist()])
            _target = [index2word[_x] for _x in target[0].tolist()]
            if bleu == "sacrebleu":
                bleu_score = sacrebleu.sentence_bleu(_prediction, _target).score
                total_val_bleu += bleu_score
            elif bleu == "nltk":
                bleu_1_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=1)
                bleu_2_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=2)
                bleu_3_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=3)
                bleu_4_score = utils.bleu(target[0].tolist(), torch.argmax(torch.stack(prediction).squeeze(1)[: target[0].shape[0], :], -1).tolist(), n=4)
                total_val_bleu_1 += bleu_1_score
                total_val_bleu_2 += bleu_2_score
                total_val_bleu_3 += bleu_3_score
                total_val_bleu_4 += bleu_4_score

            total_val_loss += loss.item()

        avg_val_loss = total_val_loss/len(val_pairs)

        if bleu == "sacrebleu":
            print(f"epoch {epoch} avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu {total_train_bleu/len(train_pairs):.2f} avg_val_bleu {total_val_bleu/len(val_pairs):.2f}")
        elif bleu == "nltk":
            print(f"epoch {epoch} avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu_1 {total_train_bleu_1/len(train_pairs):.2f} avg_train_bleu_2 {total_train_bleu_2/len(train_pairs):.2f} avg_train_bleu_3 {total_train_bleu_3/len(train_pairs):.2f} avg_train_bleu_4 {total_train_bleu_4/len(train_pairs):.2f} avg_val_bleu_1 {total_val_bleu_1/len(val_pairs):.2f} avg_val_bleu_2 {total_val_bleu_2/len(val_pairs):.2f} avg_val_bleu_3 {total_val_bleu_3/len(val_pairs):.2f} avg_val_bleu_4 {total_val_bleu_4/len(val_pairs):.2f}")

        with open(os.path.join(model_dir, f"debug_{epoch}_end.out"), "w") as h:
            if bleu == "sacrebleu":
                h.write(f"# avg_train_loss {total_train_loss/len(train_pairs)} avg_val_loss {total_val_loss/len(val_pairs)} avg_train_bleu {total_train_bleu/len(train_pairs)} avg_val_bleu {total_val_bleu/len(val_pairs)}")
            elif bleu == "nltk":
                h.write(f"# avg_train_loss {total_train_loss/len(train_pairs):.4f} avg_val_loss {avg_val_loss:.4f} avg_train_bleu_1 {total_train_bleu_1/len(train_pairs):.2f} avg_train_bleu_2 {total_train_bleu_2/len(train_pairs):.2f} avg_train_bleu_3 {total_train_bleu_3/len(train_pairs):.2f} avg_train_bleu_4 {total_train_bleu_4/len(train_pairs):.2f} avg_val_bleu_1 {total_val_bleu_1/len(val_pairs):.2f} avg_val_bleu_2 {total_val_bleu_2/len(val_pairs):.2f} avg_val_bleu_3 {total_val_bleu_3/len(val_pairs):.2f} avg_val_bleu_4 {total_val_bleu_4/len(val_pairs):.2f}")
            h.write("\n")
            h.write("\t".join(["sentence", "prediction", "attention", "fixations"]))
            h.write("\n")
            for s, input in zip(debug_sentences, utils.sent_iter(debug_sentences, word2index=word2index)):
                prediction, words, attentions, fixations = network(input)
                prediction = torch.argmax(torch.stack(prediction).squeeze(1), -1).detach().cpu().tolist()
                prediction = [index2word.get(x, "<__UNK__>") for x in prediction]
                attentions = attentions.detach().cpu().squeeze().tolist()
                fixations = fixations.detach().cpu().squeeze().tolist()
                h.write(f"{s}\t{prediction}\t{attentions}\t{fixations}")
                h.write("\n")
                
        utils.save_model(network, word2index, os.path.join(model_dir, f"{model_name}_{epoch}"))

        if best_val_loss is None or avg_val_loss < best_val_loss:
            if best_val_loss is not None:
                logger.info(f"{avg_val_loss} < {best_val_loss} ({avg_val_loss-best_val_loss}): new best model from epoch {epoch}")
            else:
                logger.info(f"{avg_val_loss} < {best_val_loss}: new best model from epoch {epoch}")

            best_val_loss = avg_val_loss
            # save_model(model, word2index, model_name + "_epoch_" + str(epoch))
            # utils.save_model(network, word2index, os.path.join(model_dir, f"{model_name}_best"))

        epoch += 1

In [2]:
import os
os.chdir(r'/kaggle/working')